In [1]:
import pandas as pd
import numpy as np

from glob import glob
import re
import string
import funcy as fp
from gensim import models
from gensim.corpora import Dictionary, MmCorpus
import nltk

In [2]:
clean_mini = pd.read_csv('clean_sql_stack_060220.csv')

In [3]:
clean_mini.head()

,question,summary,tags,user_id,no_of_votes,no_of_answers,date,clean_text
0,How to resolve related foreign key at insert t...,Motivations\n\nI have data coming from a sourc...,"['python', 'pandas', 'orm', 'sqlalchemy', 'for...",jlandercy,4.0,2.0,2019-08-02,resolve relate foreign key insert time use sql...
1,How do I match up USER table and USERINFO tabl...,I have a UserLogin table for users to login wi...,"['php', 'mysql', 'insert', 'innodb', 'php-7.2']",Jc Meyer,0.0,2.0,2019-08-02,match user table userinfo table record one one
2,FileNotFoundError [WinError 2] The system cann...,Context: I'm raising FileNotFound errors in a ...,"['python', 'sql-server', 'tkinter', 'pyodbc']",emalcolmb,0.0,0.0,2019-08-02,filenotfounderror winerror system can not find...
3,How to install TCL for the Postgres 10 docker ...,I'm trying set up pgMail for the PostgreSQL in...,['postgresql'],Nicklas Karlsson,0.0,0.0,2019-08-02,install tcl postgres docker image
4,how to fix connection leakage error in hikarip...,i have a server which i am running locally. th...,"['java', 'spring', 'postgresql', 'jpa', 'hikar...",arsalan,1.0,0.0,2019-08-02,fix connection leakage error hikaripool spring...


In [4]:
clean_mini.shape

(1280879, 8)

In [5]:
clean_mini['clean_text'].isna().sum()

5

In [6]:
clean_mini = clean_mini.dropna(subset=['clean_text'])

In [7]:
clean_mini['clean_text'] = clean_mini['clean_text'].str.split(' ')

In [8]:
clean_mini['clean_text'].head()

0    [resolve, relate, foreign, key, insert, time, ...
1    [match, user, table, userinfo, table, record, ...
2    [filenotfounderror, winerror, system, can, not...
3              [install, tcl, postgres, docker, image]
4    [fix, connection, leakage, error, hikaripool, ...
Name: clean_text, dtype: object

In [9]:
def nltk_stopwords():
    return set(nltk.corpus.stopwords.words('english'))

def prep_corpus(docs, additional_stopwords=set(), no_below=5, no_above=0.5):
  print('Building dictionary...')
  dictionary = Dictionary(docs)
  stopwords = nltk_stopwords().union(additional_stopwords)
  stopword_ids = map(dictionary.token2id.get, stopwords)
  dictionary.filter_tokens(stopword_ids)
  dictionary.compactify()
  dictionary.filter_extremes(no_below=no_below, no_above=no_above, keep_n=None)
  dictionary.compactify()

  print('Building corpus...')
  corpus = [dictionary.doc2bow(doc) for doc in docs]

  return dictionary, corpus

In [10]:
dictionary, corpus = prep_corpus(clean_mini['clean_text'])

Building dictionary...
Building corpus...


In [11]:
MmCorpus.serialize('models/sql_corpus.mm', corpus)
dictionary.save('models/sql_dict.dict')

In [12]:
%%time
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=15)
                                      
lda.save('models/sql_lda.model')

CPU times: user 12min 29s, sys: 13.2 s, total: 12min 42s
Wall time: 6min 52s


In [13]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis

In [14]:
%%time
vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

CPU times: user 42min 44s, sys: 53.9 s, total: 43min 38s
Wall time: 45min


In [16]:
%%time
p = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.save_html(p, 'sql_lda.html')

CPU times: user 36min 16s, sys: 22.9 s, total: 36min 39s
Wall time: 36min 49s
